In [1]:
# Install the necessary libraries
!pip install -U -q diffusers accelerate transformers peft bitsandbytes
!pip install -q wandb # For metric identification and tracking

# Download the official training script for SDXL/SSD-1B
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/train_text_to_image_lora_sdxl.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 35.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 5.2.0 which is incompatible.
gradio 5.49.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.5 which is incompatible.
--2026-02-19 08:46:39--  https://raw.githubusercontent.com/huggingface/diffusers/main/examples/text_to_image/train_text_to_image_l

In [2]:
# 1. Uninstall existing version to avoid conflicts
!pip uninstall -y diffusers

# 2. Install the latest "dev" version from source
!pip install git+https://github.com/huggingface/diffusers

# 3. Ensure other dependencies are up to date for SSD-1B
!pip install -U -q accelerate transformers peft bitsandbytes

Found existing installation: diffusers 0.36.0
Uninstalling diffusers-0.36.0:
  Successfully uninstalled diffusers-0.36.0
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-jicm89lz
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-jicm89lz
  Resolved https://github.com/huggingface/diffusers to commit fe78a7b7c6c0de7fd1d5ba957efc8d74ef7b00dc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.37.0.dev0-py3-none-any.whl size=4928884 sha256=08cc32afd82f7a9436c8ca752214d1b5f5aa50bcc70b3783bb97d547db6696ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-cup1qr92/wheels/90/d4/44/a58bc00fb405fefb633b0d9d2307f6e3aec6cc1775d82555d3
Successfully built diffusers


In [ ]:
# Cell 3: Data Preparation with NEAREST NEIGHBOR upscaling
import pandas as pd
import os
import json
from PIL import Image
from tqdm import tqdm

# --- CONFIGURATION ---
CSV_PATH = "/kaggle/input/labels/demo.csv" 
IMG_SOURCE_DIR = "/kaggle/input/pixel-art/images/images"
TRAIN_DIR = "/kaggle/working/train_data"
LIMIT = 3000

# Use 512 for faster training, 1024 if you have GPU memory
TARGET_SIZE = 1024  
BACKGROUND_COLOR = (255, 255, 255)  # White background

# Style prefix to add to ALL captions
STYLE_PREFIX = "pixel art sprite, crisp pixels, game asset, "

os.makedirs(TRAIN_DIR, exist_ok=True)

df = pd.read_csv(CSV_PATH)
df_sample = df.sample(n=min(LIMIT, len(df)), random_state=42)

metadata = []
print(f"Processing and Upscaling {len(df_sample)} images with NEAREST NEIGHBOR...")

for index, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
    file_name = row['file_name']
    caption = str(row['text'])
    
    source_path = os.path.join(IMG_SOURCE_DIR, file_name)
    target_filename = os.path.splitext(file_name)[0] + ".png"
    target_path = os.path.join(TRAIN_DIR, target_filename)
    
    if os.path.exists(source_path):
        try:
            # 1. Open and handle transparency
            img = Image.open(source_path).convert("RGBA")
            background = Image.new("RGBA", img.size, BACKGROUND_COLOR + (255,))
            composite = Image.alpha_composite(background, img).convert("RGB")
            
            # 2. CRITICAL: Nearest Neighbor upscaling preserves crisp pixels!
            composite = composite.resize((TARGET_SIZE, TARGET_SIZE), resample=Image.NEAREST)
            
            # 3. Save as PNG (no compression artifacts)
            composite.save(target_path, "PNG")
            
            # 4. Add style prefix to caption
            full_caption = STYLE_PREFIX + caption
            metadata.append({"file_name": target_filename, "text": full_caption})
            
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

# Save metadata
with open(os.path.join(TRAIN_DIR, "metadata.jsonl"), 'w') as f:
    for entry in metadata:
        f.write(json.dumps(entry) + "\n")

print(f"\n✅ Successfully prepared {len(metadata)} images")
print(f"📁 Saved to: {TRAIN_DIR}")
print(f"📝 Sample caption: {metadata[0]['text']}")

Moving 3000 images to training folder...


100%|██████████| 3000/3000 [00:18<00:00, 162.85it/s]

Successfully prepared 3000 images in /kaggle/working/train_data


In [ ]:
# Cell 4: Training with FIXED parameters to prevent loss collapse
!accelerate launch --num_processes=1 train_text_to_image_lora_sdxl.py \
  --pretrained_model_name_or_path="segmind/SSD-1B" \
  --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
  --train_data_dir="/kaggle/working/train_data" \
  --caption_column="text" \
  --resolution=512 \
  --mixed_precision="fp16" \
  --train_batch_size=2 \
  --gradient_accumulation_steps=4 \
  --learning_rate=5e-5 \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=100 \
  --rank=32 \
  --max_train_steps=1500 \
  --checkpointing_steps=300 \
  --validation_prompt="16x16 pixel art of a small owl flying, solid white background" \
  --validation_epochs=1 \
  --num_validation_images=4 \
  --output_dir="/kaggle/working/court_of_owls_lora_v2" \
  --seed=42 \
  --report_to="tensorboard" \
  --dataloader_num_workers=2 \
  --noise_offset=0.1

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [5]:
# import matplotlib.pyplot as plt
# from PIL import Image
# import glob
# import os

# # Check multiple possible locations for validation images
# possible_paths = [
#     '/kaggle/working/court_of_owls_lora/samples/*.png',
#     '/kaggle/working/court_of_owls_lora/*.png',
#     '/kaggle/working/court_of_owls_lora/**/*.png'
# ]

# sample_images = []
# for path in possible_paths:
#     sample_images.extend(glob.glob(path, recursive=True))

# sample_images = list(set(sample_images))  # Remove duplicates
# sample_images.sort(key=os.path.getmtime)

# if sample_images:
#     # Show the last 4 validation images
#     num_to_show = min(4, len(sample_images))
#     fig, axes = plt.subplots(1, num_to_show, figsize=(5*num_to_show, 5))
#     if num_to_show == 1:
#         axes = [axes]
    
#     for ax, img_path in zip(axes, sample_images[-num_to_show:]):
#         img = Image.open(img_path)
#         ax.imshow(img)
#         ax.axis('off')
#         ax.set_title(os.path.basename(img_path))
    
#     plt.tight_layout()
#     plt.show()
# else:
#     print("No validation images found yet.")
#     print("Check output directory contents:")
#     !ls -la /kaggle/working/court_of_owls_lora/